In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import cv2
import pickle
import threading
from lesson_functions import *
from boxes import *

with open('svm.p', 'rb') as f:
    data = pickle.load(f)
    print(data)
    svc = data['svm']
    X_scaler = data['scaler']
    params = data['params']


{'svm': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0), 'params': {'hog_feat': True, 'spatial_size': (32, 32), 'orient': 11, 'pix_per_cell': 16, 'spatial_feat': True, 'hist_feat': True, 'cell_per_block': 2, 'C': 1, 'hist_bins': 32, 'hog_channel': 'ALL', 'kernel': 'linear', 'color_space': 'LUV'}, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}


In [ ]:
import time
import cv2
from skimage.feature import hog
from boxes import *

class WindowFeatureThread(threading.Thread):
    def __init__(self, img_cs, windows):
        threading.Thread.__init__(self)
        
        self.windows = windows
        self.window_features = []
        self.img_cs = np.copy(img_cs)
        
    def run(self):
        for window in self.windows:
            window_img = self.img_cs[window[0][1]:window[1][1], window[0][0]:window[1][0]]
            window_img_64 = cv2.resize(window_img, (64,64))

            single_window_features = single_img_features(window_img_64, 
                        spatial_size=params['spatial_size']
                        hist_bins=params['hist_bins'], 
                        orient=parms['orient'], 
                        pix_per_cell=params['pix_per_cell'], 
                        cell_per_block=params['cell_per_block'],
                        hog_channel=params['hog_channel'],
                        spatial_feat=params['spatial_feat'],
                        hist_feat=params['hist_feat'], 
                        hog_feat=True,
                        vis=False)
            
            self.window_features.append(single_window_features)
                            
class WindowFeatureFinder():
    def __init__(self, img_cs):
        self.img_cs = img_cs
        self.windows = get_all_windows(img_cs)
    
    # returns features for all windows
    def find_features(self):
        threads = []

        t1 = time.time()
        per_thread = len(self.windows) // 10

        for i in range(0, len(boxes), per_thread):
            if i + per_thread < len(boxes):
                box_range = boxes[i:i + per_thread]
            else:
                box_range = boxes[i:]

            thread = CarDetector(img_cs, box_range)
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()
            full_hog.extend(thread.box_hogs)

        t2 = time.time()
        print('Total time taken for {} boxes to be reduced and hogged {:.2f} secs'.format(len(boxes), t2 - t1))